# Importing Relevant Libraries

In [7]:
import numpy as np
from numpy.random import default_rng

from gpcam import GPOptimizer

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import os
import csv

from scipy.stats import entropy

import random

import torch
from torch import nn

# Loading the Data

In [20]:
energy_data_total = np.load("/data/Dataset 1/energy.npy")
cycle_number_total = np.load("/data/Dataset 1/cycle.npy")


# use slicing to select every other column and all rows
energy_data = energy_data_total[:, ::2]
cycle_number = cycle_number_total[::2]

num_of_datasets = len(energy_data[:,0])

label_size = 30
plt.figure(figsize = (20,10))
for i in range(len(energy_data)): plt.scatter(cycle_number,energy_data[i]) 

plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.xlabel("Cycle Number",fontsize=label_size) 
plt.ylabel("Energy",fontsize=label_size)
plt.show()


# Creating the cycle_data array with the same shape of energy_data, where all rows have the same values as cycle_number_range. This results in 1 row repeated 22 times.
#cycle_data = np.full_like(energy_data, cycle_number_range)

print("max y: ", np.max(energy_data))

max y:  363.5404228825251


# Creating the Subfolder in Results

In [1]:
# Specify the path for the new folder
new_folder_path = f"/results/Dataset 1 Figures/With Stopping/"

# Create the folder
os.makedirs(new_folder_path, exist_ok=True)



NameError: name 'os' is not defined

# Deep Kernel Learning

In [9]:
# All remaining code asssumes that the NN architecture is made up of two hidden layers and same number of nodes
# If other architectures are used, the indices of the hyperparameters and their boudsn need to be changed accordingly
# Number of nodes can be varied as the user prefer

class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.nodes_num = 5

        # Inputs to hidden layer linear transformation
        self.layer1 = nn.Linear(1, self.nodes_num)
        self.layer2 = nn.Linear(self.nodes_num, self.nodes_num)
        self.layer3 = nn.Linear(self.nodes_num, 1)

    def forward(self, x):
        # Pass the input tensor through each of our operations
        #print(x)
        x = torch.Tensor(x)
        x = torch.nn.functional.relu(self.layer1(x))
        x = torch.nn.functional.relu(self.layer2(x))
        x = torch.nn.functional.relu(self.layer3(x))
        return x.detach().numpy()

    def set_weights(self,w1,w2,w3):
      with torch.no_grad():
        self.layer1.weight = nn.Parameter(torch.from_numpy(w1).float())
        self.layer2.weight = nn.Parameter(torch.from_numpy(w2).float())
        self.layer3.weight = nn.Parameter(torch.from_numpy(w3).float())

    def set_biases(self,b1,b2,b3):
      with torch.no_grad():
        self.layer1.bias = nn.Parameter(torch.from_numpy(b1).float())
        self.layer2.bias = nn.Parameter(torch.from_numpy(b2).float())
        self.layer3.bias = nn.Parameter(torch.from_numpy(b3).float())

    def get_weights(self):
        return self.layer1.weight, self.layer2.weight, self.layer3.weight
    def get_biases(self):
        return self.layer1.bias, self.layer2.bias, self.layer3.bias

n = Network()

# Defining the GP Components

In [10]:
# For the squared Exponential Function
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d)

def my_noise(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    my_slope     = hps[total_num_of_NN_hps+1]
    my_pow       = hps[total_num_of_NN_hps+2]
    my_intercept = hps[total_num_of_NN_hps+3]

    my_s = my_slope * x**my_pow + my_intercept
    noise = np.diag(my_s[:,0])
    
    return noise

# Kernel Function
def kernel_nn(x1,x2,hps,obj):

    nodes_num           = obj.args[0]
    total_num_of_NN_hps = obj.args[1]

    trained_NN_hps      = obj.args[3:][0]
    
    # NN
    n.set_weights(trained_NN_hps[0:nodes_num].reshape(nodes_num,1),
                  trained_NN_hps[nodes_num:nodes_num**2+nodes_num].reshape(nodes_num,nodes_num),
                  trained_NN_hps[nodes_num**2+nodes_num:nodes_num**2+2*nodes_num].reshape(1,nodes_num))

    n.set_biases(trained_NN_hps[nodes_num**2+2*nodes_num:nodes_num**2+3*nodes_num].reshape(nodes_num),
                 trained_NN_hps[nodes_num**2+3*nodes_num:nodes_num**2+4*nodes_num].reshape(nodes_num),
                 np.array([trained_NN_hps[nodes_num**2+4*nodes_num]]))

    x1_nn = n.forward(x1).reshape(-1,1)
    x2_nn = n.forward(x2).reshape(-1,1)
    d = get_distance_matrix(x1_nn,x2_nn)


    # Main Function
    k = hps[total_num_of_NN_hps] * obj.squared_exponential_kernel(d, 50) #100

    return k


################################################################################

# Mean function: Two-Element piecewise function
def mean2(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    x0 = hps[total_num_of_NN_hps+4]
    
    m1 = hps[total_num_of_NN_hps+5]
    m2 = hps[total_num_of_NN_hps+6]

    b1  = obj.args[2]
    b2 = (m1 - m2) * x0 + b1

    x = x[:,0]

    y = np.where(x <= x0, m1*x + b1, m2*x + b2)
                
    return y

# Creating Distribution

In [11]:
def make_distribution(m,v,x_pred):
    distr = np.zeros((len(x_pred),1000))
    y = np.linspace(0,360,1000)
    std = np.sqrt(v)
    for i in range(len(x_pred)):
        distr[i] = 1./(std[i] * np.sqrt(2 * np.pi)) * np.exp( - (y - m[i])**2 / (2. * std[i]**2))
    return distr

# Finding Ground Truth Probability of Failure

In [12]:
data_size = 20

x_data_all = np.tile(cycle_number, data_size).reshape(-1, 1)   # repeat cycle 20 times to create x_data
y_data_all = np.vstack(energy_data[0:20,:]).reshape(-1, 1)                  # stack energy rows to create y_data


nodes_num = n.nodes_num

total_num_of_NN_hps = 0 # nodes_num**2 + 4*nodes_num + 1     # Depends on the number of layers used
num_of_other_hps = 7                                     # Depends on Kernel, noise and mean functions


init_hyperparameters = np.array([1500,              # Kernel
                           0.06, 2, 2,             # Noise  
                           250,-0.01,-0.015])      # Mean.


# Trained NN hyperparameters
#     These hyperparameters were trained a priori to lower the computational requirements during the battery testing sequence
#     The user has the choice to train these NN hyperparameters with those of the Kernel, Noise, and mean functions
NN_weights_trained_hps = np.array([0.157, 0.149, 0.029, 0.061, 0.016, 0.219, 0.436, 0.148, 0.461, 1.772, 1.945, 0.084,
                                   1.743, 0.731, 1.881, 1.342, 1.015, 0.982, 1.795, 0.4,   1.062, 0.753, 1.052, 1.252,
                                   1.557, 0.708, 0.149, 1.348, 1.481, 0.568, 0.028, 0.036, 0.107, 0.05,  0.101])

NN_biases_trained_hps = np.array([1.168, 1.915, 0.184, 0.145, 0.893, 0.377, 1.289, 0.911, 1.971, 0.339, 1.573])

my_trained_NN_hps = np.concatenate([NN_weights_trained_hps,NN_biases_trained_hps])



In [13]:
dx = 1000./1000.
  
# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((total_num_of_NN_hps + num_of_other_hps,2))

# Kernel Sq Exp 
bounds[total_num_of_NN_hps] = np.array([500.,7000.])                             # Kernel Variance
#bounds[total_num_of_NN_hps+7] = np.array([10.,300.])                           # Kernel Lengthscale

# Noise
bounds[total_num_of_NN_hps+1] = np.array([1e-5,15.])                           # Noise Slope
bounds[total_num_of_NN_hps+2] = np.array([1,5.])                            # Noise Power
bounds[total_num_of_NN_hps+3] = np.array([0.,5.])                              # Noise Intercept
# Mean
bounds[total_num_of_NN_hps+4] = np.array([200.,450.])                          # Mean Piecewise Intersection point
bounds[total_num_of_NN_hps+5] = np.array([-1e-1,-1e-3])                        # Mean Slope 1
bounds[total_num_of_NN_hps+6] = np.array([-1e-1,-1e-2])                        # Mean Slope 2


trained_hps = np.array([ 3.69988606e+03,  1.00111228e-05,  2.96700098e+00,  2.25831837e+00,
  2.04048171e+02, -2.78484076e-02, -9.82988265e-02])


    
# Finding the mean of the initial capacity to be entered to the code
my_ind = np.where(x_data_all<=10)
initial_capacity = np.mean(y_data_all[my_ind[0]])
    
my_gpGT = GPOptimizer(x_data_all,y_data_all,
            #init_hyperparameters = init_hyperparameters,  
            init_hyperparameters = trained_hps,  
            compute_device='cpu', 
            gp_kernel_function=kernel_nn, 
            gp_kernel_function_grad=None, 
            gp_mean_function=mean2, 
            gp_mean_function_grad=None,
            gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))

 
#my_gpGT.train(hyperparameter_bounds=bounds, method='global')

my_GT_hps = my_gpGT.hyperparameters

print("GT Training Done!")

GT Training Done!


/tmp/ipykernel_3366/2048398358.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


In [21]:
print("GT Hyperparameters: ", my_gpGT.hyperparameters)

x_pred = np.linspace(0,600,1001).reshape(-1,1)

f_GT = my_gpGT.posterior_mean(x_pred.reshape(-1,1))["f(x)"]
v_GT = my_gpGT.posterior_covariance(x_pred.reshape(-1,1), variance_only=False, add_noise=True)["v(x)"]


# Plotting the data
plt.figure(figsize = (15,10))
plt.plot(x_pred[:,0],f_GT, color = "red", linewidth = 3,label='Fitted Model')
plt.fill_between(x_pred[:,0],f_GT - 2. * np.sqrt(v_GT), f_GT + 2. * np.sqrt(v_GT), alpha = 0.5, color = "grey", label='Confidence Interval')
plt.scatter(x_data_all,y_data_all, color = "blue",s = 250, label='Data Points') # Training Data
plt.xlabel("Cycle Number",fontsize=label_size) 
plt.ylabel("Energy",fontsize=label_size)
plt.ylim(150,370)
plt.yticks([150, 200, 250, 300, 350])
plt.xlim(0,600)
plt.xticks([0,150,300,450,600])
plt.legend(fontsize=label_size,frameon=False,loc='lower left')
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes\plt.savefig('/results/Dataset 1 Figures/With Stopping/POF_my_plot_GT.png') # saving plot with a unique name 
plt.savefig('/results/Dataset 1 Figures/With Stopping/Ground Truth Fit.png') # saving plot with a unique name 

GT Hyperparameters:  [ 3.69988606e+03  1.00111228e-05  2.96700098e+00  2.25831837e+00
  2.04048171e+02 -2.78484076e-02 -9.82988265e-02]


In [22]:
d_GT = make_distribution(f_GT,v_GT,x_pred)[:,800].T

d_GT = d_GT/(np.sum(d_GT) * dx)


# Number cycles where 5% of batteries failed 
cum_distr = np.cumsum(d_GT)*dx
index_95failed = np.argmin(abs(cum_distr - 0.05))
print("95 percent failed after: ", x_pred[index_95failed,0])
  
epsilon_for_failure_distribution = 1e-10
d_GT = np.maximum(d_GT, epsilon_for_failure_distribution)

x_pred1 = np.append(x_pred,np.array([600.1,700]))
d_GT1 = np.append(d_GT,np.array([1e-10,1e-10]))

#print(d.shape,x_pred[:,0].shape)
plt.figure(figsize = (10,10))
plt.plot(x_pred1, d_GT1,linewidth = 5)
plt.xlabel("cycle number",fontsize=label_size)
plt.ylabel(" ",fontsize=label_size)
plt.axvline(x=x_pred[index_95failed,0], color='red', linestyle='--',linewidth = 5)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.fill_between(x_pred1, d_GT1, where=(x_pred1 <= x_pred[index_95failed, 0]),interpolate=True, color='red', alpha=0.3)
plt.yticks([])
plt.xlim([200,700])
plt.savefig('/results/Dataset 1 Figures/With Stopping/POF_my_plot_GT.png') # saving plot with a unique name 
#plt.close() # closing the plot to free up memory

95 percent failed after:  331.2


# __________________________________________________________________
# Sequence of Experiments 
# Multiple Trials, Different batteries sequence

In [16]:
# Function to calculate the difference between the thr All-Data GP and a one-Battery GP

def current_battery_GP_calculations(cycle_number, energy_data, test_battery, battery, 
                                    my_trained_hps, kernel, mean2, my_noise, initial_capacity, x_data_old ,y_data_old,
                                    my_calc_kl,error_threshold , current_battery):

    my_battery_stopping_condition = False

    x_data_current_battery = cycle_number[0:test_battery+1]
    y_data_current_battery = energy_data[battery,0:test_battery+1]
    
    my_gp_current_battery = GPOptimizer(x_data_current_battery,y_data_current_battery,
                         init_hyperparameters = my_trained_hps,  # we need enough of those for kernel, noise and prior mean functions
                         #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
                         compute_device='cpu', 
                         gp_kernel_function=kernel_nn, 
                         gp_kernel_function_grad=None, 
                         gp_mean_function=mean2, 
                         gp_mean_function_grad=None,
                         gp_noise_function=my_noise,
                         normalize_y=False,
                         sparse_mode=False,
                         gp2Scale = False,
                         store_inv=False, 
                         ram_economy=False, 
                         args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))
    
    
    my_gp_Old_with_current_battery = GPOptimizer(x_data_old,y_data_old,
                         init_hyperparameters = my_trained_hps,  # we need enough of those for kernel, noise and prior mean functions
                         #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
                         compute_device='cpu', 
                         gp_kernel_function=kernel_nn, 
                         gp_kernel_function_grad=None, 
                         gp_mean_function=mean2, 
                         gp_mean_function_grad=None,
                         gp_noise_function=my_noise,
                         normalize_y=False,
                         sparse_mode=False,
                         gp2Scale = False,
                         store_inv=False, 
                         ram_economy=False, 
                         args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))
        

    
    ####################################################
    x_new_lower_limit = cycle_number[test_battery]
    
    x_pred2 = np.linspace(x_new_lower_limit,600,50).reshape(-1,1)
    
    my_posterior_mean_CurrentBattery = my_gp_current_battery.posterior_mean(x_pred2)["f(x)"]
    my_posterior_covariance_CurrentBattery = my_gp_current_battery.posterior_covariance(x_pred2)['S'] + 1e-7 * np.eye(len(x_pred2)) 
    
    my_posterior_mean_AllData = my_gp_Old_with_current_battery.posterior_mean(x_pred2)["f(x)"]
    my_posterior_covariance_AllData = my_gp_Old_with_current_battery.posterior_covariance(x_pred2)['S'] + 1e-7 * np.eye(len(x_pred2))
    
    diff_CurrentBattery_AllData = my_gp_Old_with_current_battery.kl_div(my_posterior_mean_CurrentBattery,my_posterior_mean_AllData,my_posterior_covariance_CurrentBattery,my_posterior_covariance_AllData)

    ########################################################################

    
    my_calc_kl = np.append(my_calc_kl,diff_CurrentBattery_AllData)
    
    # Calculate the cumulative sum up to the current index
    cumulative_sum = np.sum(my_calc_kl)
    my_thresholded_average = error_threshold * cumulative_sum / len(my_calc_kl)
        
    
    if diff_CurrentBattery_AllData < my_thresholded_average and test_battery > 2:
        print("diff_CurrentBattery_AllData: ", diff_CurrentBattery_AllData)
        print("my_thresholded_average: ", my_thresholded_average)
        print("Battery ", current_battery, "  is not informative, it is SKIPPED")
        my_battery_stopping_condition = True
    
    
    return np.array([diff_CurrentBattery_AllData, my_thresholded_average,my_calc_kl,my_battery_stopping_condition])

In [18]:
x_pred = np.linspace(0,1500,1001).reshape(-1,1)


init_hyperparameters = my_GT_hps


label_size = 30
error_threshold = 0.5 #Threshold used to stop the experiments. A value of 0 means no stopping criterion is implemented


num_versions = 30

# Set the random seed for reproducibility
random_seed = 42
# Generate 10 different random sequences
np.random.seed(random_seed)  # Set the seed outside the loop
random_sequences = [np.random.permutation(20) for _ in range(num_versions)]

#for t in range(0,num_versions):
for t in range(6,7):

    # Specify the path for the new folder
    new_folder_path = f"/results/Dataset 1 Figures/With Stopping/Trial_{t}"

    # Create the folder
    os.makedirs(new_folder_path, exist_ok=True)

    %cd {new_folder_path}
    
    # Check if the csv files exist and delete them if present
    if os.path.exists(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Failure_Cycle_Number.csv"): os.remove(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Failure_Cycle_Number.csv")
    if os.path.exists(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Trained Hyperparameters.csv"): os.remove(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Trained Hyperparameters.csv")    
    if os.path.exists(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Total Correlation.csv"): os.remove(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Total Correlation.csv")
        
        
    # battery Sequence
    my_sequence_of_batteries = random_sequences[t]
    counter_for_sequence_of_batteries = 4

    battery_A = my_sequence_of_batteries[0]
    battery_B = my_sequence_of_batteries[1]
    battery_C = my_sequence_of_batteries[2]
    battery_D = my_sequence_of_batteries[3]

    # Initializing the battery tests
    test_battery_A = 0
    test_battery_B = 0
    test_battery_C = 0
    test_battery_D = 0

    stop_all_battery_A = False
    stop_all_battery_B = False
    stop_all_battery_C = False
    stop_all_battery_D = False

    # Do I retrain the GP Model everytime I add a point
    condition_for_GP_training = True
    my_calc_kl = np.empty((0, 1))

    x_data = np.empty((0, 1))
    y_data = np.empty((0, 1))

    my_counter = 0


    while True:

        if my_counter% 2 == 0: 
            print("My Counter: ", my_counter)
            print("A: ", battery_A,  test_battery_A)
            print("B: ", battery_B,  test_battery_B)
            print("C: ", battery_C,  test_battery_C)
            print("D: ", battery_D,  test_battery_D)


        stop_this_battery_A = False
        stop_this_battery_B = False
        stop_this_battery_C = False
        stop_this_battery_D = False

        # Adding the required experimental data

        # If there is still data to be added
        if not(stop_all_battery_A):
            battery_A_x_data = cycle_number[test_battery_A:test_battery_A+1]
            battery_A_y_data = energy_data[battery_A,test_battery_A:test_battery_A+1] 
        else: # Otherwise
            battery_A_x_data = np.empty((0))
            battery_A_y_data = np.empty((0))    

        # If there is still data to be added
        if not(stop_all_battery_B):
            battery_B_x_data = cycle_number[test_battery_B:test_battery_B+1]
            battery_B_y_data = energy_data[battery_B,test_battery_B:test_battery_B+1]
        else: # Otherwise
            battery_B_x_data = np.empty((0))
            battery_B_y_data = np.empty((0))


        # If there is still data to be added
        if not(stop_all_battery_C):
            battery_C_x_data = cycle_number[test_battery_C:test_battery_C+1]
            battery_C_y_data = energy_data[battery_C,test_battery_C:test_battery_C+1]
        else: # Otherwise
            battery_C_x_data = np.empty((0))
            battery_C_y_data = np.empty((0))

        # If there is still data to be added
        if not(stop_all_battery_D):
            battery_D_x_data = cycle_number[test_battery_D:test_battery_D+1]
            battery_D_y_data = energy_data[battery_D,test_battery_D:test_battery_D+1]
        else: # Otherwise
            battery_D_x_data = np.empty((0))
            battery_D_y_data = np.empty((0))

        x_to_append = np.concatenate((battery_A_x_data,battery_B_x_data,battery_C_x_data,battery_D_x_data))
        y_to_append = np.concatenate((battery_A_y_data,battery_B_y_data,battery_C_y_data,battery_D_y_data))    

        # Break the while loop when there are no more tests to be done
        if np.size(x_to_append) == 0:
            break

        # To be used for the GP of All Data when comparing the expected information gain,
        # without the batteries being tested now, except for the one we are interested in
        x_data_old = np.array(x_data)
        y_data_old = np.array(y_data)

        x_data = np.append(x_data, x_to_append)    
        y_data = np.append(y_data, y_to_append)   


        #########################################################################
        #########################################################################
        #########################################################################
        # Main GP With All Data

        # Finding the mean of the initial capacity to be entered to the code
        my_ind = np.where(x_data<=10)
        initial_capacity = np.mean(y_data[my_ind[0]])

        my_gp_All = GPOptimizer(x_data,y_data,
                                     init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
                                     #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
                                     compute_device='cpu', 
                                     gp_kernel_function=kernel_nn, 
                                     gp_kernel_function_grad=None, 
                                     gp_mean_function=mean2, 
                                     gp_mean_function_grad=None,
                                     gp_noise_function=my_noise,
                                     normalize_y=False,
                                     sparse_mode=False,
                                     gp2Scale = False,
                                     store_inv=False, 
                                     ram_economy=False, 
                                     args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


        if condition_for_GP_training:
            my_gp_All.train(hyperparameter_bounds=bounds, method='global')

        my_trained_hps = my_gp_All.hyperparameters

        # Saving Trained Hyperparameters
        row_hyperparameters = [my_counter] +  list(my_trained_hps)

        # Write the row_entropies to the CSV file
        csv_file = open(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Trained Hyperparameters.csv", 'a', newline='')
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(row_hyperparameters)
        csv_file.close() # Close the CSV file


        f = my_gp_All.posterior_mean(x_pred)["f(x)"]
        v =  my_gp_All.posterior_covariance(x_pred, variance_only=False, add_noise=True)["v(x)"]
        vnn =  my_gp_All.posterior_covariance(x_pred, variance_only=False, add_noise=False)["v(x)"]      

         # Plotting the data
        plt.figure(figsize = (10,10))
        plt.plot(x_pred[:,0],f, color = "red", linewidth = 5,label=' ')
        plt.fill_between(x_pred[:,0],f - 2. * np.sqrt(v), f + 2. * np.sqrt(v), alpha = 0.4, color = "grey", label=' ')
        plt.scatter(x_data,y_data, s=250, color = "blue",label=' ') # Training Data
        #plt.xlabel("Cycle Number",fontsize=label_size) 
        #plt.ylabel("Energy",fontsize=label_size)
        plt.xlabel(" ",fontsize=label_size) 
        plt.ylabel(" ",fontsize=label_size)
        plt.ylim(150,370)
        #plt.yticks([150, 200, 250, 300, 350])
        plt.yticks([])
        plt.xlim(0,600)
        #plt.xticks([0,150,300,450,600])
        plt.xticks([])
        plt.legend(fontsize=label_size,frameon=False,loc='lower left')
        plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
        plt.savefig(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/my_plot_{my_counter}.pdf", bbox_inches='tight')
        plt.close() 
       

        #########################################################################
        #########################################################################
        #########################################################################

        # Calculate Probability of Failure Distribution and finding the cycle at which 5% failed <90% initial energy 

        d = make_distribution(f,v,x_pred)[:,800].T
        d = d/(np.sum(d) * dx)

        # Number cycles where 5% of batteries failed 
        cum_distr = np.cumsum(d)*dx
        index_95failed = np.argmin(abs(cum_distr - 0.05))

        d = np.maximum(d, epsilon_for_failure_distribution)

        # Calculating the KL divergence between the current probability of failure and the GT probability of failrue
        my_kl_between_pof = entropy(d, d_GT)

        row_failures = [my_counter, x_pred[index_95failed,0], my_kl_between_pof]
        csv_file = open(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Failure_Cycle_Number.csv", 'a', newline='')
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(row_failures)
        csv_file.close() # Close the CSV file


        if my_kl_between_pof<=0.05:
            print("No Training")
            condition_for_GP_training = False

        my_counter +=1


        #########################################################################
        #########################################################################
        #########################################################################

        # Stop the battery that is not informative

        if my_counter > 50 : # Start checking after the first set of 4 batteries are done
            diff_CurrentBattery_AllData_A = 0
            diff_CurrentBattery_AllData_B = 0
            diff_CurrentBattery_AllData_C = 0
            diff_CurrentBattery_AllData_D = 0

            # Battery A
            if not(stop_all_battery_A):
                [diff_CurrentBattery_AllData_A, my_thresholded_average_A,my_calc_kl,stop_this_battery_A] = current_battery_GP_calculations(cycle_number, energy_data, test_battery_A, battery_A, 
                                                                                                 my_trained_hps, kernel_nn, mean2, my_noise, initial_capacity, x_data_old ,y_data_old,
                                                                                                 my_calc_kl,error_threshold,"A") 
            # Battery B
            if not(stop_all_battery_B):
                [diff_CurrentBattery_AllData_B, my_thresholded_average_B,my_calc_kl,stop_this_battery_B] = current_battery_GP_calculations(cycle_number, energy_data, test_battery_B, battery_B, 
                                                                                                 my_trained_hps, kernel_nn, mean2, my_noise, initial_capacity, x_data_old ,y_data_old,
                                                                                                  my_calc_kl,error_threshold,"B")
            # Battery C   
            if not(stop_all_battery_C):
                [diff_CurrentBattery_AllData_C, my_thresholded_average_C,my_calc_kl,stop_this_battery_C] = current_battery_GP_calculations(cycle_number, energy_data, test_battery_C, battery_C, 
                                                                                                 my_trained_hps, kernel_nn, mean2, my_noise, initial_capacity, x_data_old ,y_data_old,
                                                                                                  my_calc_kl,error_threshold,"C")
            # Battery D  
            if not(stop_all_battery_D):
                [diff_CurrentBattery_AllData_D, my_thresholded_average_D,my_calc_kl,stop_this_battery_D] = current_battery_GP_calculations(cycle_number, energy_data, test_battery_D, battery_D, 
                                                                                                 my_trained_hps, kernel_nn, mean2, my_noise, initial_capacity, x_data_old ,y_data_old,
                                                                                                  my_calc_kl,error_threshold,"D")



            # Create a row with the extracted values
            row_entropies = [my_counter, diff_CurrentBattery_AllData_A, diff_CurrentBattery_AllData_B, 
                         diff_CurrentBattery_AllData_C, diff_CurrentBattery_AllData_D]

            # Write the row_entropies to the CSV file
            csv_file = open(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Total Correlation.csv", 'a', newline='')
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(row_entropies)
            csv_file.close() # Close the CSV file

        #########################################################################
        #########################################################################
        #########################################################################


        # Update for Battery A
        test_battery_A = test_battery_A + 1

        # If current battery A finished testing or If Battery A is stopped      
        if test_battery_A > 49 or stop_this_battery_A:     
            if counter_for_sequence_of_batteries<20:
                battery_A = my_sequence_of_batteries[counter_for_sequence_of_batteries]
                counter_for_sequence_of_batteries +=1
                test_battery_A = 0
            else:
                stop_all_battery_A = True

        #############################
        # Update for Battery B
        test_battery_B = test_battery_B + 1

        # If current battery B finished testing or If Battery B is stopped      
        if test_battery_B > 49 or stop_this_battery_B:
            if counter_for_sequence_of_batteries<20:
                battery_B = my_sequence_of_batteries[counter_for_sequence_of_batteries]
                counter_for_sequence_of_batteries +=1
                test_battery_B = 0
            else:
                stop_all_battery_B = True

        #############################   
        # Update for Battery C
        test_battery_C = test_battery_C + 1

        # If current battery C finished testing or If Battery A is stopped      
        if test_battery_C > 49 or stop_this_battery_C:
            if counter_for_sequence_of_batteries<20:
                battery_C = my_sequence_of_batteries[counter_for_sequence_of_batteries]
                counter_for_sequence_of_batteries +=1
                test_battery_C = 0
            else:
                stop_all_battery_C = True

        ############################# 
        # Update for Battery D
        test_battery_D = test_battery_D + 1

        # If current battery D finished testing or If Battery D is stopped      
        if test_battery_D > 49 or stop_this_battery_D:
            if counter_for_sequence_of_batteries<20:
                battery_D = my_sequence_of_batteries[counter_for_sequence_of_batteries]
                counter_for_sequence_of_batteries +=1
                test_battery_D = 0
            else:
                stop_all_battery_D = True       

        #############################   


        print("===========================")


    print("Training Complete for All Batteries")

    # Save to .npy file
    np.save(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Training_X_data.npy", x_data)
    np.save(f"/results/Dataset 1 Figures/With Stopping/Trial_{t}/Training_Y_data.npy", y_data)
    
    
    %cd ..
    
    row_num_experiments = [t, x_data.shape[0]]
    # Write the number of experiments done in this trial to the CSV file
    csv_file = open(f"/results/Dataset 1 Figures/With Stopping/Number of Experiments.csv", 'a', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(row_num_experiments)
    csv_file.close() # Close the CSV file

print("All Trials are done")

/results/Dataset 1 Figures/With Stopping/Trial_6
My Counter:  0
A:  13 0
B:  10 0
C:  12 0
D:  16 0


/tmp/ipykernel_3366/329689771.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


/tmp/ipykernel_3366/329689771.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


My Counter:  2
A:  13 2
B:  10 2
C:  12 2
D:  16 2


/tmp/ipykernel_3366/329689771.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


/tmp/ipykernel_3366/329689771.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


My Counter:  4
A:  13 4
B:  10 4
C:  12 4
D:  16 4


/tmp/ipykernel_3366/329689771.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args=np.array([nodes_num,total_num_of_NN_hps,initial_capacity,my_trained_NN_hps]))


KeyboardInterrupt: 